In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import os
from pydicom import dcmread
import pydicom
import re
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
import sys
import time
from functools import partial





# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def sorted_alphanumeric(data):  #find this in stackoverflow 
    
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    
    return sorted(data, key=alphanum_key)
    #sort list 


def search(Path , dir , mri_type): 
    
    lstFilesDCM = []  
    for root, dirs, files in os.walk(f"{Path}/{dir}/{mri_type}", topdown=False):
        for name in files:
            lstFilesDCM.append(os.path.join(root, name))

    return (lstFilesDCM) #return list of the image in a dir

In [ ]:
def _bytes_feature(value):         # THX tensorflow doc
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def load_dicom_image(path):
    img_size = (256, 256)
    data = pydicom.read_file(path).pixel_array.astype(dtype = 'float32', copy = False)
    image = cv2.resize(data , img_size , interpolation=cv2.INTER_LANCZOS4) 
    
    return image


def Build_entire_train_dataset_V3(Mri_type, Path):
    Csv  = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")

    
    Csv = Csv.set_index("BraTS21ID")
    unwanted = [109, 123,709]
    Csv = Csv.drop(labels = [e for e in unwanted] , axis = 0)
    
    df = Csv["MGMT_value"]
    
    

    img_size = (256,256)
    depth = 400
    start = time.time()


    train_dir = os.listdir(Path)    
    
    unwanted = ['00109', '00123', '00709']
    train_dir = [e for e in train_dir if e not in unwanted]

    
    train_dir = sorted_alphanumeric(train_dir)
    print(len(train_dir))     # 582 
    
    option = tf.io.TFRecordOptions(compression_level=2, compression_type="ZLIB")

    with tf.io.TFRecordWriter(f"Train_{Mri_type}.tfrec", options=option) as writer:
        for dir,y in zip(train_dir,df):

            Path_list = search(Path, dir , Mri_type)

            sorted_path = sorted_alphanumeric(Path_list)

            image3d = np.stack([load_dicom_image( f) for f in sorted_path], axis=0)

            if image3d.shape[0] < depth:
                n_zero = np.zeros((depth-image3d.shape[0], 256 , 256 )) 
                image3d = np.concatenate((image3d, n_zero), axis=0 ).astype(dtype='float32')
                image3d = np.expand_dims(image3d , axis=-1)
                image3d = image3d/4096

            data = {'image': _bytes_feature(image3d.tobytes()),
                    'label': _int64_feature(y)
                    }

            Data = tf.train.Example(features=tf.train.Features(feature=data))
            Data = Data.SerializeToString()

            writer.write(Data)
            print(dir)
            print(y)

    elapsed = time.time()
    elapsed = elapsed - start
    print("Time spent: ", elapsed)

In [ ]:
if __name__ == "__main__":
    Path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"
    Build_entire_train_dataset_V3("FLAIR", Path)
